In [28]:
import json
import pandas as pd
import re
import requests

In [29]:
def load_jsonl(path):
    data = []
    with open(path, 'r', encoding='utf-8') as reader:
        for line in reader:
            data.append(json.loads(line)) 
        return data  


In [10]:
data = load_jsonl('theft_sex_gram_100.jsonl')

In [20]:

def tokenize_news(row):
    regex_pattern = "\[::ID::\](.*)\[::Date::\](.*)\[::TITLE::\](.*)\[::INTRO::\](.*)\[::DESC::\](.*)"
    text = row['text'].replace("\n", " ")
    m = re.search(regex_pattern, text).groups()
    news_id, publish_date, news_source, news_title, news_intro, news_description = m[0], m[1], row['Source'], m[2], m[3], m[4]
    return news_id, publish_date, news_source, news_title, news_intro, news_description
    

    

def tokenize_news_info(row):
    news_info_metadata = ["criminal", "action", "victim", "trigger",
                          "location", "datetime", "rootcause", "worth", "item", "enforcement"]
    
    regex_pattern = "\[::ID::\](\w+)"
    text = row['text']
    m = re.search(regex_pattern, text).groups()
    news_id = m[0]
    l = { 
        "news_id" : news_id, "criminal" : [], "action" : [], "victim" : [],
        "trigger" : [],"location" : [], "datetime" : [], "rootcause" : [],
        "worth" : [],"item" : [],"enforcement" : []
        }
   
    
    for label in row['label']:
        if label[2].lower() in news_info_metadata:
            start = label[0]
            end = label[1]
            l[label[2].lower()].append(text[start:end])

    return news_id, l['criminal'], l['action'], l['victim'], l['trigger'], l['location'], l['datetime'], l['rootcause'], l['worth'], l['item'], l['enforcement']


def transform_data(data):
    news = {'news_id' : [],
                 'publish_date' : [],
                 'news_source' : [],
                 'news_title' : [],
                 'news_intro' : [],
                 'news_description' : []}

    news_info = {'news_id' : [],
                 'criminal' : [],
                 'action' : [],
                 'victim' : [],
                 'trigger' : [],
                 'location' : [],
                 'datetime' : [] ,
                 'rootcause' : [],
                 'worth' : [],
                 'item' : [],
                 'enforcement' : []}

    for row in data:
        try:
            news_id, publish_date, news_source, news_title, news_intro, news_description = tokenize_news(row)
            news_id, criminal, action, victim, trigger, location, datetime, rootcause, worth, item, enforcement  = tokenize_news_info(row)
            news['news_id'].append(news_id)
            news['publish_date'].append(publish_date)
            news['news_source'].append(news_source)
            news['news_title'].append(news_title)
            news['news_intro'].append(news_intro)
            news['news_description'].append(news_description)
           
            news_info['news_id'].append(news_id)
            news_info['criminal'].append(criminal)
            news_info['action'].append(action)
            news_info['victim'].append(victim)
            news_info['trigger'].append(trigger)
            news_info['location'].append(location)
            news_info['datetime'].append(datetime)
            news_info['rootcause'].append(rootcause)
            news_info['worth'].append(worth)
            news_info['item'].append(item)
            news_info['enforcement'].append(enforcement)
                
            
        except Exception as e: 
            print(e)
            print(row['id'])
        
        
    return news, news_info
    

def get_location(df):
    for index, row in df.iterrows():
        print(row['location'])
        
        
# params = {
#     'key' : API_KEY,
#     'keyword' : 'จ.พิษณุโลก',
#     'limit' : 1
# }

# base_url = 'https://search.longdo.com/mapsearch/json/search?'
# response = requests.get(base_url, params=params).json()
# # response.raise_for_status()
# response.keys()

# #if respones['hasmore'] == False:
    
    

In [21]:
news , news_info = transform_data(data)

In [22]:
news = pd.DataFrame(news)
news_info = pd.DataFrame(news_info)

In [24]:
get_location(news_info)

['อ.โพธาราม']


In [26]:
news

,news_id,publish_date,news_source,news_title,news_intro,news_description
0,THR_1866929,12 มิ.ย. 2563 01:10 น.,ไทยรัฐ,โรคจิตจนมุม ถูกรวบคาห้องเช่า ย่องฉก กกน.สาว ซว...,หนุ่มหื่นจนมุม ถูกรวบคาห้องเช่า ย่องลัก กกน.สา...,หนุ่มหื่นจนมุม ถูกรวบคาห้องเช่า ย่องลัก กกน.สา...
1,THR_1894804,22 ก.ค. 2563 08:27 น.,ไทยรัฐ,"รวบแล้ว ""โจรลักหลับ"" ปลอมเป็นตร. ปลดทรัพย์เหยื...","ชุดสืบสวนสน.สายไหม แกะรอยรวบ ""โจรลักหลับ"" ใช้ค...","ชุดสืบสวนสน.สายไหม แกะรอยรวบ ""โจรลักหลับ"" ใช้ค..."
2,THR_1918867,27 ส.ค. 2563 08:55 น.,ไทยรัฐ,ศาลอาญานัดชี้ชะตา อดีต ผอ.กอล์ฟ ฆ่าโหด 3 ศพ ชิ...,ศาลอาญานัดชี้ชะตาอดีต ผอ.กอล์ฟ จากเหตุคดีสะเทื...,ศาลอาญานัดชี้ชะตาอดีต ผอ.กอล์ฟ จากเหตุคดีสะเทื...
3,THR_1990619,8 ธ.ค. 2563 00:09 น.,ไทยรัฐ,โจรบุกเดี่ยวชิงทอง 4 บาท รัวปืนยิงฝ่าประตูกระจ...,โจรสวมชุดขายไก่ย่างดังร้านดังเมืองคอน ควงปืนฉา...,โจรสวมชุดขายไก่ย่างดังร้านดังเมืองคอน ควงปืนฉา...
4,THR_1866071,11 มิ.ย. 2563 11:54 น.,ไทยรัฐ,"กองปราบ คุม ""ลุงบุญช่วย-ลูกชาย"" ฝากขังผัดแรก ค...",จนท.กองปราบปราม เบิกตัว ลุงบุญช่วยและลูกชาย ผู...,จนท.กองปราบปราม เบิกตัว ลุงบุญช่วยและลูกชาย ผู...
...,...,...,...,...,...,...
346,THR_1905567,7 ส.ค. 2563 12:10 น.,ไทยรัฐ,"ผบช.สตม.แจง หมายจับ ""บอส"" ของตร.สากลยังอยู่ เข...",ผบช.สตม.แจง คดีบอส สน.ทองหล่อ มีหนังสือแจ้งว่า...,ผบช.สตม.แจง คดีบอส สน.ทองหล่อ มีหนังสือแจ้งว่า...
347,THR_1908444,11 ส.ค. 2563 18:36 น.,ไทยรัฐ,"""เนตร นาคสุข"" รองอัยการสูงสุด สั่งไม่ฟ้องคดีบอ...","""เนตร นาคสุข"" รองอัยการสูงสุด สั่งไม่ฟ้องคดี ""...","""เนตร นาคสุข"" รองอัยการสูงสุด สั่งไม่ฟ้องคดี ""..."
348,THR_1903089,4 ส.ค. 2563 08:09 น.,ไทยรัฐ,เปิดภาพในที่เกิดเหตุ ยิงสนั่นบ่อนย่านพระราม 3 ...,เปิดภาพที่เกิดเหตุ ยิงกันในบ่อนย่านพระราม 3 พบ...,เปิดภาพที่เกิดเหตุ ยิงกันในบ่อนย่านพระราม 3 พบ...
349,THR_2005817,4 ม.ค. 2564 04:42 น.,ไทยรัฐ,ล่ามือปืนประกบยิงช่างซ่อม จยย.อดีตผู้ต้องขังคด...,เจ้าของร้านซ่อมจักรยานยนต์ในเขตดินแดง ถูกมือปื...,เจ้าของร้านซ่อมจักรยานยนต์ในเขตดินแดง ถูกมือปื...


In [27]:
news_info

,news_id,criminal,action,victim,trigger,location,datetime,rootcause,worth,item,enforcement
0,THR_1866929,[นายธนพล ทองเมือง หรือพีท อายุ 20 ปี],[ลักลอบขโมย],[เจ้าของร้านมินิมาร์ท],[นำกางเกงชั้นในสตรีมาสูดดม],[อ.โพธาราม],[23.51 น. ของวันที่ 19 พ.ค.63],[ทุกครั้งที่เห็นกางเกงในของผู้หญิงที่ตนชอบ จะเ...,[],[ชุดชั้นในสตรี ],[พ.ต.อ.สมมาตร จันทรัตน์ ผกก.สภ.โพธาราม พร้อมด้...
1,THR_1894804,[นายจรูญ หรือทวน รอดเกษม อายุ 53 ปี],[ปลดทรัพย์],[นายชัยรัตน์ พุ่มทับทิม อายุ 46 ปี ],[ปลุกให้ตื่น ถ้าไม่ตื่นก็ปลดทรัพย์],"[เขตสายไหม, พื้นที่ใกล้เคียง]",[ตุลาคมปี 62],[ผู้เสียหายนอนหลับในรถยนต์],[],[],[ พ.ต.อ.ทนงศิลป์ มณีโชติ ผกก.สน.สายไหม กทม.]
2,THR_1918867,[นายประสิทธิชัย เขาแก้ว],[ฆ่าและพยายามฆ่า],[],[],[ห้างโรบินสันลพบุรี ],[],[],"[ผู้เสียชีวิต 3 ราย บาดเจ็บหลายราย, มูลค่าราว ...","[อาวุธปืน ขนาด 9มม, ทองคำ 31 เส้น]",[]
3,THR_1990619,[],[ปล้น],"[ร้านทอง, พนักงานขายของร้านทอง, น.ส.ณัฐกานต์ ร...",[],[ถนน-นาหลวง เลขที่ 118 ถนนนครศรีธรรมราช-ปากพนั...,"[ 18.00 น. วันที่ 7 ธ.ค. 2563, 17.30 น.]",[],[ประตูกระจกแตกเสียหาย],[ทองคำหนัก 2 บาท 2 เส้น],[พ.ต.ท.ดนัย ชลมุณี สว.(สอบสวน) สภ.ชะเมา อ.เมือ...
4,THR_1866071,"[นายบุญช่วย เจริญสถาพร อายุ 80 ปี, นายกิตติพงษ...",[ยักยอกที่ดิน],[มูลนิธิอภิธรรมมหาธาตุ],[],[],[],[],[],"[3,800 ไร่]",[กองบังคับการปราบปราม(บก.ป.) เจ้าหน้าที่ตำรวจ ...
...,...,...,...,...,...,...,...,...,...,...,...
346,THR_1905567,[นายวรยุทธ หรือบอส อยู่วิทยา ],[],[],[],[],[],[],[],[],[]
347,THR_1908444,"[""บอส อยู่วิทยา"" ]",[],[],[],[],[],[],[],[],[]
348,THR_1903089,[],[ถืออาวุธปืนเข้ามาข่มขู่ทวงเงินคืน โดยอ้างว่าถ...,[ผู้เสียชีวิต จำนวน 4 ราย เป็นชาย 2 ราย และหญิ...,[],[บ่อนแห่งหนึ่งย่านพระราม 3 ซอย 66 แขวงช่องนนทร...,[],[],[],[],[]
349,THR_2005817,[นายสืบศักดิ์ สุภาพชน หรือ ตั้ม อายุ 44 ปี เจ้...,[ขับขี่เข้ามาประกบก่อนชักปืนยิงใส่ นายสืบศักดิ...,[],[การทะเลาะวิวาทซึ่งหน้า],[ใกล้ร้านซ่อมรถจักรยานยนต์ไม่มีชื่อ ท้ายซอยประ...,[ 01.30 น. วันที่ 4 ม.ค. 2564],[],[],[],[ร.ต.อ.ปภาวิน ดอนมูลไพร รอง สว.(สอบสวน) สน.ดิน...
